<a href="https://colab.research.google.com/github/madhirv/HDS5210-Assignments/blob/main/midterm/midterm-2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, resp_rate, ox_sat, bpm, sys_BP, temp, alert, insp_oxy, perf_status):
    """
    doctest
    >>> priest('male', 21, 12, .55, 45, 100, 37.1, 'Alert', 'Air', 'bed/chair bound, no self-care')
    0.34
    >>> priest('female', 65, 8, .95, 60, 124, 37.9, 'Alert', 'Supplemental Oxygen', 'limited activity, can self-care')
    0.29
    >>> priest('male', 12, 2, .90, 132, 100, 15, 'supplemental oxygen', 'Air', 'limited strenuous activity, can do light activity')
    0.55
    """

    points = 0

    sex = sex.lower()
    if sex == 'male':
        points += 1
    elif sex != 'female':
        return None

    #age
    if 50 <= age <= 65:
        points += 2
    elif 66 <= age <= 80:
        points += 3
    elif age > 80:
        points += 4

    #respiratory rate
    if resp_rate < 9 or resp_rate > 24:
        points += 3
    elif 9 <= resp_rate <= 11:
        points += 1
    elif 21 <= resp_rate <= 24:
        points += 2

    #oxygen saturation
    if ox_sat == .94 or ox_sat == .95:
        points += 1
    elif .92 <= ox_sat <= .93:
        points += 2
    elif ox_sat < .92:
        points += 3

    #bpm
    if bpm < 41 or bpm > 130:
        points += 3
    elif 41 <= bpm <= 50 or 91 <= bpm <= 110:
        points += 1
    elif 111 <= bpm <= 130:
        points += 2

    #Systolic blood pressure
    if sys_BP < 91 or sys_BP > 219:
        points += 3
    elif 91 <= sys_BP <= 100:
        points += 2
    elif 101 <= sys_BP <= 110:
        points += 1

    #Temperature
    if temp < 35.1:
        points += 3
    elif 35.1 <= temp <= 36.0 or 38.1 <= temp <= 39.0:
        points += 1
    elif temp > 39.0:
        points += 2

    #Points for alertness status
    if alert.lower() == 'confused or not alert':
        points += 3

    # Points for inspiration oxygen
    if insp_oxy.lower() == 'supplemental oxygen':
        points += 2

    #Points for performance status
    performance_scores = {
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
        'bed/chair bound, no self-care': 4
    }
    points += performance_scores.get(perf_status.lower(), 0)

    thresholds = [(2, 0.01), (4, 0.02), (5, 0.03), (6, 0.09), (7, 0.15), (8, 0.18),
                  (9, 0.22), (10, 0.26), (11, 0.29), (12, 0.34), (13, 0.38), (14, 0.46),
                  (15, 0.47), (16, 0.49), (17, 0.55), (25, 0.59)]

    for threshold, value in thresholds:
        if points < threshold:
            return value
    return 0.99


In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('male', 21, 12, .55, 45, 100, 37.1, 'Alert', 'Air', 'bed/chair bound, no self-care')
Expecting:
    0.34
ok
Trying:
    priest('female', 65, 8, .95, 60, 124, 37.9, 'Alert', 'Supplemental Oxygen', 'limited activity, can self-care')
Expecting:
    0.29
ok
Trying:
    priest('male', 12, 2, .90, 132, 100, 15, 'supplemental oxygen', 'Air', 'limited strenuous activity, can do light activity')
Expecting:
    0.55
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
def find_hospital(age,sex,risk):
  """
  doctest examples:
  >>> find_hospital(72,'male', 0.65)
  'Wesley Woods Geriatric Hospital'
  >>> find_hospital(50,'female', 0.30)
  'Select Specialty Hospital - Northeast Atlanta'
  >>> find_hospital(50,'male', 0.85)
  'Emory Dunwoody Medical Center'
  """
  url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd"
  parameters = {
    "age": age,
    "sex": sex.lower(),
    "risk_pct" : risk
  }
  response = requests.get(url, params=parameters)
  if response.status_code == 200:
    data = response.json()
    return data['hospital']
  else:
    return None

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(72,'male', 0.65)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(50,'female', 0.30)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(50,'male', 0.85)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
def get_address(hospital_name):
  """
  doctest examples:
  >>> get_address('Wesley Woods Geriatric Hospital')
  '1821 CLIFTON ROAD, NE'
  >>> get_address('GWINNETT HOSPITAL SYSTEM')
  '1000 MEDICAL CENTER BOULEVARD'
  >>> get_address('DORMINY MEDICAL CENTER')
  '200 PERRY HOUSE ROAD, BOX 1447'
  """

  hospital_name = hospital_name.upper()
  response = requests.get('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')
  if response.status_code == 200:
    data = response.json()
  else:
    return None
  return data[hospital_name]['ADDRESS']


In [6]:
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address('GWINNETT HOSPITAL SYSTEM')
Expecting:
    '1000 MEDICAL CENTER BOULEVARD'
ok
Trying:
    get_address('DORMINY MEDICAL CENTER')
Expecting:
    '200 PERRY HOUSE ROAD, BOX 1447'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [7]:
import requests

url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'

def process_people(url):
    response = requests.get(url)
    data = response.text.split('\n')

    Final = {}

    for line in data[1:]:  # Skips the first line
        if len(line) < 10:
            continue
        # Unpack and process fields
        pname, sex, age, resp_rate, ox_sat, bpm, sys_BP, temp, alert, insp_oxy, perf_status = line.split('|')
        age, resp_rate, bpm, sys_BP = map(int, [age, resp_rate, bpm, sys_BP])
        ox_sat, temp = map(float, [ox_sat, temp])

        # Calculate risk and hospital info
        risk = priest(sex, age, resp_rate, ox_sat, bpm, sys_BP, temp, alert, insp_oxy, perf_status)
        hname = find_hospital(age, sex, risk)
        address = get_address(hname)

        # Store final details in dictionary
        Final[pname] = [sex, age, resp_rate, ox_sat, bpm, sys_BP, temp, alert, insp_oxy, perf_status, risk, hname, address]

    return Final


In [8]:
output = process_people(url)
output

{'E9559': ['FEMALE',
  40,
  24,
  0.96,
  105,
  115,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  51,
  19,
  0.99,
  90,
  91,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  40,
  29,
  0.96,
  105,
  95,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  66,
  19,
  0.96,
  135,
  115,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'Wesley Woods Geriatric Hospital',
  '1821 CLIFTON ROAD, NE'],
 'E8661': ['MALE',
  18,
  22,
  0.95,
  105,
  115,
  36.3,
  'Confused or Not Alert',
  'air',
  'unrestricted normal activity',
  0.22,
  'Southwest Hospital and Medical Center',
  '50

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [9]:
import json
url = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'
response = requests.get(url)
data = response.json()

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---